# Formula analysis

This notebook analyzes behavior of formulas from the EFD in various exceptional cases, such as operating on the point at infinity or the all zero point they sometimes degenerate into.

In [21]:
import io
import json
import random

from pprint import pprint
import tabulate
from IPython.display import HTML, display

from pyecsca.ec.params import load_params_ecgen
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.point import Point
from pyecsca.ec.mod import mod
from pyecsca.ec.error import UnsatisfiedAssumptionError
from pyecsca.misc.cfg import TemporaryConfig

In [2]:
model = ShortWeierstrassModel()
affine_coords = AffineCoordinateModel(model)

curve_data = b"""[{
    "field": {
        "p": "0x57880ae612d14d33afd0c965938ac1ba44824036cea5d4a1699a9f44000fb273"
    },
    "a": "0x55d376d1fbcc919da841bb13352d4e419ac85a100fb806014bed884db5916399",
    "b": "0x21e27f7a065039ee59fd6b12c33d96709642aa6ac3738bd4f66fc663c79a19f8",
    "order": "0x57880ae612d14d33afd0c965938ac1b91f16808ee875095bafed41e136ca7bfe",
    "subgroups": [
        {
            "x": "0x407d5c52d9ad6f25bd7ff25f07804b4e4ebd4f5c992eafeb8c92e33f81e73b85",
            "y": "0x4b92eefcfa7c5e295c7e649801b83649156974064a8649f9a94f915754bd2183",
            "order": "0x57880ae612d14d33afd0c965938ac1b91f16808ee875095bafed41e136ca7bfe",
            "cofactor": "0x1",
            "points": [
                {
                    "x": "0x2571326cc99fe050bfe1a6a02ea635c56504e49d122152fd281761748a0501d9",
                    "y": "0x0000000000000000000000000000000000000000000000000000000000000000",
                    "order": "0x2"
                },
                {
                    "x": "0x266a5c5927e4f6feec30a9f3e2acb535657f365e1a24c1bb0b0d9158a7668639",
                    "y": "0x49c431e2a2704efb4b193e0fa26c60f815eaf195f712befd53b7bafb72b98488",
                    "order": "0x2bc405730968a699d7e864b2c9c560dc8f8b4047743a84add7f6a0f09b653dff"
                }
            ]
        }
    ]
}]"""
curve_json = json.loads(curve_data)[0]
p = int(curve_json["field"]["p"], 16)
order2_aff = Point(affine_coords,
                   x=mod(int(curve_json["subgroups"][0]["points"][0]["x"], 16), p),
                   y=mod(int(curve_json["subgroups"][0]["points"][0]["y"], 16), p))
orderbig_aff = Point(affine_coords,
                     x=mod(int(curve_json["subgroups"][0]["points"][1]["x"], 16), p),
                     y=mod(int(curve_json["subgroups"][0]["points"][1]["y"], 16), p))

def allzero(pt):
    return all(value == 0 for value in pt.coords.values())

def affine(pt):
    try:
        pt.to_affine()
    except Exception:
        return False
    return True

def on_curve(curve, pt):
    try:
        return curve.is_on_curve(pt)
    except Exception:
        return False

def eval_test(expected, out, curve):
    return (expected.equals_homog(out) if expected is not None else "Undefined", allzero(out), affine(out), on_curve(curve, out), out)

In [26]:
results_add = {}
results_dbl = {}

with TemporaryConfig() as cfg:
    cfg.ec.unsatisfied_formula_assumption_action = "ignore"
    for coords_name, coords in model.coordinates.items():
        try:
            params = load_params_ecgen(io.BytesIO(curve_data), coords_name, infty=False)
        except UnsatisfiedAssumptionError:
            print(f"Skipping {coords_name}, unsatisfied assumption")
            continue
        results_add[coords_name] = {}
        results_dbl[coords_name] = {}
        infty = params.curve.neutral
        order2 = order2_aff.to_model(coords, params.curve)
        orderbig = orderbig_aff.to_model(coords, params.curve)
        orderbig_neg = params.curve.affine_negate(orderbig_aff).to_model(coords, params.curve)
        orderbig2 = params.curve.affine_double(orderbig_aff).to_model(coords, params.curve)
        r1_aff = params.curve.affine_random()
        r1 = r1_aff.to_model(coords, params.curve)
        r2_aff = params.curve.affine_add(order2_aff, r1_aff)
        r2 = r2_aff.to_model(coords, params.curve)
        r1r2_aff = params.curve.affine_add(r1_aff, r2_aff)
        r1r2 = r1r2_aff.to_model(coords, params.curve)
        zeros = Point(coords, **{var: mod(0, p) for var in coords.variables})
        random.seed(hash(r1_aff) + hash(r2_aff))
        garbage = Point(coords, **{var: mod(random.randint(0, p), p) for var in coords.variables})

        adds = set(formula for formula in coords.formulas.values() if formula.shortname == "add")
        dbls = set(formula for formula in coords.formulas.values() if formula.shortname == "dbl")
        for add in adds:
            res = {}
            results_add[coords_name][add.name] = res
            # P + P = ?
            PpP = add(p, orderbig, orderbig, **params.curve.parameters)[0]
            # P + infty = ?
            PpInfty = add(p, orderbig, infty, **params.curve.parameters)[0]
            InftypP = add(p, infty, orderbig, **params.curve.parameters)[0]
            # ord2 + ord2 = ?
            O2pO2 = add(p, order2, order2, **params.curve.parameters)[0]
            # P + Q = infty
            EqInfty1 = add(p, orderbig, orderbig_neg, **params.curve.parameters)[0]
            EqInfty2 = add(p, orderbig_neg, orderbig, **params.curve.parameters)[0]
            # P + zeros = ?
            PpZeros = add(p, orderbig, zeros, **params.curve.parameters)[0]
            ZerospP = add(p, zeros, orderbig, **params.curve.parameters)[0]
            # P1 + P2 = ? such that P1 - P2 = ord2
            PpQord2 = add(p, r1, r2, **params.curve.parameters)[0]
            # P + garbage = ?
            PpGarbage = add(p, orderbig, garbage, **params.curve.parameters)[0]
            GarbagepP = add(p, garbage, orderbig, **params.curve.parameters)[0]
            res["PpP"] = eval_test(orderbig2, PpP, params.curve)
            res["PpInfty"] = eval_test(orderbig, PpInfty, params.curve)
            res["InftypP"] = eval_test(orderbig, InftypP, params.curve)
            res["O2pO2"] = eval_test(infty, O2pO2, params.curve)
            res["EqInfty1"] = eval_test(infty, EqInfty1, params.curve)
            res["EqInfty2"] = eval_test(infty, EqInfty2, params.curve)
            res["PpZeros"] = eval_test(None, PpZeros, params.curve)
            res["ZerospP"] = eval_test(None, ZerospP, params.curve)
            res["PpQDifford2"] = eval_test(r1r2, PpQord2, params.curve)
            res["PpGarbage"] = eval_test(None, PpGarbage, params.curve)
            res["GarbagepP"] = eval_test(None, GarbagepP, params.curve)
        for dbl in dbls:
            res = {}
            results_dbl[coords_name][dbl.name] = res
            O2twice = dbl(p, order2, **params.curve.parameters)[0]
            Inftytwice = dbl(p, infty, **params.curve.parameters)[0]
            Zerostwice = dbl(p, zeros, **params.curve.parameters)[0]
            Garbagetwice = dbl(p, garbage, **params.curve.parameters)[0]
            res["O2twice"] = eval_test(infty, O2twice, params.curve)
            res["Inftytwice"] = eval_test(infty, Inftytwice, params.curve)
            res["Zerostwice"] = eval_test(None, Zerostwice, params.curve)
            res["Garbagetwice"] = eval_test(None, Garbagetwice, params.curve)

Skipping jacobian-3, unsatisfied assumption
Skipping w12-0, unsatisfied assumption
Skipping projective-3, unsatisfied assumption
Skipping jacobian-0, unsatisfied assumption
Skipping projective-1, unsatisfied assumption
Skipping xyzz-3, unsatisfied assumption


## Results (add)
 - `P + P = ?`: Two behavior classes, complete formulas (RCB) and incomplete formulas.
   For the complete, the result is correct, can be made affine and is on curve.
   For the incomplete formulas the result is zeros, not affine and not on the curve.
 - `P + infty = P` and `infty + P = P`: Four behavior classes, **for some the order matters**:
   most of the `madd`s and `zadd`s. This is because they have an assumption `Z2 = 1`. Some
   `madd`s fail the same in both cases.
    > Not correct, zeros,     not affine, not on curve
    
    > Correct,     not zeros, affine,     on curve
    
    > Not correct, not zeros, affine,     not on curve
    
    > Not correct, not zeros, not affine, not on curve
 - `Ord2 + Ord2 = infty`: Two behavior classes. Either correct behavior or zeros. Correct behavior for four fomulas on projective coords:
   `add-2002-bj, add-2007-bl, add-2015-rcb, madd-2015-rcb`
 - `P + -P = infty` and `-P + P = infty`: All correct, no zeros, no affine, on curve.
 - `zeros + P = ?` and `P + zeros = ?`: Three behavior classes, in one, the zeros propagate, in another the formula makes up an affine point that is not zeros but is not on the curve. In the final class, the formula makes up some point that is neither zeros, nor affine nor on the curve. These cases happen for the `madd`/`zadd` variants.
 - `P + Q = ?, where P - Q = Ord2`: The exceptional case for (otherwise complete) RCB formulas.
 - `P + garbage = ?` and `garbage + P = ?`:

In [28]:
table = [["Coords", "Formula", "Test", "Correct?", "Zeros?", "Affine?", "On curve?"]]
test_filter = None
groups = {}
clusters = {}
clustering_tests = ("PpP", "PpInfty", "InftypP","PpQDifford2")
for coords_name, vals in results_add.items():
    for name, formula in vals.items():
        ident = set()
        for k, v in formula.items():
            if k in clustering_tests:
                ident.add((k, v[0])) #, v[1], v[2], v[3]
            if test_filter is None or k in test_filter:
                item = (v[0], v[1], v[2], v[3])
                group = groups.setdefault(item, set())
                group.add(name + "-" + k + "-" + coords_name)
                table.append((coords_name, name, k, v[0], v[1], v[2], v[3]))
        ident = frozenset(ident)
        cluster = clusters.setdefault(ident, set())
        cluster.add(name)
for ident, cluster in clusters.items():
    print(cluster)
    print("\t", sorted(ident))
print(len(clusters))
if test_filter is not None:
    for group, formulas in groups.items():
        print(group)
        for f in sorted(formulas):
            print(f"\t{f}")
display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

{'madd-1998-cmo', 'madd', 'add-1986-cc', 'madd-2009-bl', 'mmadd-2009-bl', 'add-1998-cmo', 'add-2007-bl', 'add-2009-bl', 'madd-2008-g', 'add-2001-b', 'add-1998-hnm', 'zadd-2007-m', 'madd-2004-hmv', 'madd-2008-s', 'madd-2007-bl', 'mmadd-2008-s', 'mmadd-1998-cmo', 'mmadd-2007-bl', 'add-2008-s', 'add-1998-cmo-2'}
	 [('InftypP', False), ('PpInfty', False), ('PpP', False), ('PpQDifford2', True)]
{'add-2015-rcb'}
	 [('InftypP', True), ('PpInfty', True), ('PpP', True), ('PpQDifford2', False)]
{'add-2002-bj', 'add-2007-bl'}
	 [('InftypP', False), ('PpInfty', False), ('PpP', True), ('PpQDifford2', True)]
{'madd-2015-rcb'}
	 [('InftypP', True), ('PpInfty', False), ('PpP', True), ('PpQDifford2', False)]
4


Coords,Formula,Test,Correct?,Zeros?,Affine?,On curve?
jacobian,add-1998-cmo-2,PpP,False,True,False,False
jacobian,add-1998-cmo-2,PpInfty,False,True,False,False
jacobian,add-1998-cmo-2,InftypP,False,True,False,False
jacobian,add-1998-cmo-2,O2pO2,False,True,False,False
jacobian,add-1998-cmo-2,EqInfty1,True,False,False,True
jacobian,add-1998-cmo-2,EqInfty2,True,False,False,True
jacobian,add-1998-cmo-2,PpZeros,Undefined,True,False,False
jacobian,add-1998-cmo-2,ZerospP,Undefined,True,False,False
jacobian,add-1998-cmo-2,PpQDifford2,True,False,True,True
jacobian,add-1998-cmo-2,PpGarbage,Undefined,False,True,False


## Results (dbl)
 - `2 * Ord2 = infty`: All correct, no zeros, no affine, on curve.
 - `2 * infty = infty`: Three behavior classes: Some formulas are correct and return infty.
   Some make up some affine point that is not zeros and not on curve.
   Some return zeros.
 - `2 * zeros = ?`: Two behavior classes, the zeros either propagate or become the infty.

In [29]:
table = [["Formula", "Test", "Correct?", "Zeros?", "Affine?", "On curve?"]]
test_filter = None
groups = {}
for coords_name, vals in results_dbl.items():
    for name, formula in vals.items():
        for k, v in formula.items():
            if test_filter is None or k in test_filter:
                item = (v[0], v[1], v[2], v[3])
                group = groups.setdefault(item, set())
                group.add(name + "-" + k + "-" + coords_name)
                table.append((coords_name, name, k, v[0], v[1], v[2], v[3]))
if test_filter is not None:
    for group, formulas in groups.items():
        print(group)
        for f in sorted(formulas):
            print(f"\t{f}")
display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))

,Formula,Test,Correct?,Zeros?,Affine?,On curve?
projective,mdbl-2007-bl,O2twice,True,False,False,True
projective,mdbl-2007-bl,Inftytwice,False,False,True,False
projective,mdbl-2007-bl,Zerostwice,Undefined,False,False,True
projective,dbl-2015-rcb,O2twice,True,False,False,True
projective,dbl-2015-rcb,Inftytwice,True,False,False,True
projective,dbl-2015-rcb,Zerostwice,Undefined,True,False,False
projective,dbl-1998-cmo,O2twice,True,False,False,True
projective,dbl-1998-cmo,Inftytwice,False,True,False,False
projective,dbl-1998-cmo,Zerostwice,Undefined,True,False,False
projective,dbl-2007-bl,O2twice,True,False,False,True
